### 1.导入数据

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn import preprocessing

In [2]:
# 读取数据
testdata = pd.read_csv('../data/dataannalysis/happiness_test_abbr.csv')
traindata = pd.read_csv('../data/dataannalysis/happiness_train_abbr.csv')

#### 1.2处理数据

In [3]:
#异常数据处理
traindata = traindata[traindata['happiness'] != -8]
traindata = traindata[traindata['depression'] != -8]
traindata = traindata[traindata['class'] != -8]
traindata = traindata[traindata['health'] != -8]
traindata = traindata[traindata['equity'] != -8]
traindata = traindata[traindata['family_status'] != -8]
traindata = traindata[traindata['health_problem'] != -8]
traindata = traindata[traindata['relax'] != -8]
traindata = traindata[traindata['learn'] != -8]
traindata['view'].replace(-8,3,inplace = True)
traindata = traindata[traindata['socialize'] != -8]
traindata = traindata[traindata['socialize'] != -1]
traindata = traindata[traindata['socialize'] != -2]
traindata = traindata[traindata['socialize'] != -3]
traindata = traindata[traindata['socialize'] != 50]
traindata = traindata[traindata['status_3_before'] != -8]
traindata = traindata[traindata['status_peer'] != -8]
traindata['inc_ability'].replace(-8,3,inplace = True)

#数据重新处理
traindata['marital'].replace(2,1,inplace = True)
traindata['marital'].replace(3,1,inplace = True)
traindata['marital'].replace(4,1,inplace = True)
traindata['marital'].replace(7,1,inplace = True)
traindata['marital'].replace(6,2,inplace = True)
traindata['marital'].replace(5,3,inplace = True)

bins = [-100,0,100000,300000,500000,1000000,10000000] 
traindata['income'] = pd.cut(traindata['income'],bins, labels=['1','2', '3', '4', '5', '6'])



avg_income=traindata['family_income']/traindata['family_m']
traindata['avg_income'] = avg_income
bins = [-100,0,20000,50000,100000,1000000,10000000] 
traindata['avg_income'] = pd.cut(traindata['avg_income'],bins, labels=['1','2', '3', '4', '5', '6'])

In [4]:
#traindata['avg_income'].('unknown', inplace=True)
traindata['avg_income'] = traindata['avg_income'].cat.add_categories(['0']);
traindata['avg_income'].fillna('0', inplace=True)

In [7]:
le = sklearn.preprocessing.LabelEncoder() 
le.fit(['1','2', '3', '4', '5', '6', '0']) 
traindata['avg_income']  = le.transform(traindata['avg_income'] ) 

In [8]:
le = sklearn.preprocessing.LabelEncoder() 
le.fit(['1','2', '3', '4', '5', '6']) 
traindata['income']  = le.transform(traindata['income'] ) 

In [9]:
#重新做相关性分析
traindata.corr()['happiness']

id                 0.008267
happiness          1.000000
survey_type       -0.035656
province          -0.021285
city              -0.023442
county            -0.021499
gender             0.018517
birth             -0.004821
nationality       -0.035595
religion           0.006780
religion_freq      0.010455
edu                0.104231
income             0.043733
political          0.080010
floor_area         0.044722
height_cm          0.040638
weight_jin         0.084260
health             0.246918
health_problem     0.212923
depression         0.320490
hukou              0.079546
socialize          0.079689
relax              0.123843
learn              0.123454
equity             0.284537
class              0.301481
work_exper         0.008198
work_status       -0.039226
work_yr           -0.018063
work_type          0.021679
work_manage       -0.039622
family_income      0.052113
family_m           0.059978
family_status      0.306640
house              0.092656
car               -0

In [10]:
traindatafinal = traindata[['happiness','depression','depression','class','health','equity','family_status','health_problem','relax','learn','view','socialize','status_peer','status_3_before','inc_ability','car','avg_income','edu']]

In [11]:
train_data = traindatafinal[:7000]
test_data = traindatafinal[7000:]
 
train_data_X = train_data.drop(['happiness'],axis=1)
train_data_Y = train_data['happiness']
test_data_X = test_data.drop(['happiness'],axis=1)
test_data_Y = test_data['happiness']

### 2.定义所需函数

In [12]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
 
    return A, cache
 
def sigmoid_backward(dA, cache):
    Z = cache
 
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
 
    assert (dZ.shape == Z.shape)
 
    return dZ
 
def relu(Z):
 
    A = np.maximum(0,Z)
    cache = Z 
    return A, cache
 
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
 
    assert (dZ.shape == Z.shape)
 
    return dZ
 

### 3.初始化参数

In [13]:
def initialize_parameters_deep(layer_dims):

    np.random.seed(3)  #用于指定随机数生成时所用算法开始的整数值
    parameters = {}
    L = len(layer_dims)            # 网络的层数

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        ### END CODE HERE ###

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))


    return parameters

### 4.参数调整

#### 4.1前向传播

In [23]:
def linear_activation_forward(A_prev, W, b, activation):
    Z = np.dot(W, A_prev) + b
    linear_cache = (A_prev, W, b)
    if activation == "sigmoid":
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###

    elif activation == "relu":
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    cache = (linear_cache, activation_cache)

    return A, cache

In [24]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network

    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], "relu")
        caches.append(cache)
        ### END CODE HERE ###

    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], "sigmoid")
    caches.append(cache)
    ### END CODE HERE ###
    return AL, caches

#### 4.2损失函数

In [25]:

def compute_cost(AL, Y):
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = -np.sum(np.multiply(np.log(AL),Y) + np.multiply(np.log(1 - AL), 1 - Y)) / m
    ### END CODE HERE ###

    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    return cost

#### 4.3反向传播
    o对于第L层我们需要

        dW[l] = 1/m*dZ[l]A[l-1].T    
        db[l] = 1/m*ΣdZ[l][i]   
        dA[l-1] = W[l].T*dZ[l]
    o定义激活函数的梯度下降函数(relu_backward/sigmoid_backward) dZ= relu_backward(dA)
    o综合以上两个步骤 [LINEAR->ACTIVATION] 反向传播方法得到 dW ,db,dA
    o循环执行[LINEAR->RELU] L-1  次( 1 到L-1层) 在最后一层执行 [LINEAR->SIGMOID]

In [26]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = relu_backward(dA, activation_cache)
        ### END CODE HERE ###

    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = sigmoid_backward(dA, activation_cache)
        ### END CODE HERE ###
    A_prev, W, b = linear_cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = np.dot(dZ, A_prev.T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [27]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")

    for l in reversed(range(L-1)):
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

#### 4.4 修改参数

In [28]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    ### END CODE HERE ###
    return parameters

#### 4.5合并

In [29]:
def optimize(parameters,X,Y,num_iterations,learning_rate,print_cost):
    costs = []
    for i in range(num_iterations):
        #FORWARD PROPAGATION
        AL, caches = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        if i % 100 == 0:
            costs.append(cost)
        if print_cost and i % 100 == 0:
            print("Cost after interation %i:%f"%(i,cost))

        # BACKWARD PROPAGATION (TO FIND GRAD)
        grads = L_model_backward(AL, Y, caches)
        # update rule
        parameters = update_parameters(parameters, grads, learning_rate)
    return parameters,costs

### 5.模型

In [30]:
def n_layer_model(X,Y,layers_dims,learning_rate=0.0075,num_iterations=3000,print_cost=False,isPlot=True):
    np.random.seed(1)
    grads = {}
    costs = []
    (n_x,n_h,n_y) = layers_dims

    """
    初始化参数
    """
    parameters = initialize_parameters_deep(layers_dims)

    """
    开始进行迭代
    """
    parameters,costs= optimize(parameters,X,Y,num_iterations,learning_rate,print_cost)
        
        
    #迭代完成，根据条件绘制图
    if isPlot:
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

    #返回parameters
    return parameters

### 6.测试
假设我们的是一个两层模型，输入参数是，输出参数是，隐藏层一层

In [ ]:
train_data_X.shape 

In [31]:
n_x = 17
n_h = 7
n_y = 4
layers_dims = (n_x,n_h,n_y)

parameters = n_layer_model(train_data_X, train_data_Y, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True,isPlot=True)


#test_data_X test_data_Y

ValueError: shapes (7,10) and (7000,17) not aligned: 10 (dim 1) != 7000 (dim 0)

In [ ]:
W[l]:(n[l],n[l-1])  
b[l]:(n[l],1)  
dW[l]:(n[l],n[l-1])  
db[l]:(n[l],1)  
Z[l]:(n[l],1)  
A[l]:(n[l],1)  